In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.preprocessing.image import load_img, img_to_array
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
import random
import numpy as np
import pandas as pd

### Download File Zip

In [ ]:
!wget --no-check-certificate \
  https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip`

/bin/bash: -c: line 0: unexpected EOF while looking for matching ``'
/bin/bash: -c: line 1: syntax error: unexpected end of file


### Ekstraksi pada file zip

In [ ]:
import zipfile,os
local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
labels = ['paper','scissors','rock']
nb = len(labels)

### Membagi Gambar dan Label

In [ ]:
def input_target_split(train_dir,labels):
    dataset = []
    count = 0
    for label in labels:
        folder = os.path.join(train_dir,label)
        for image in os.listdir(folder):
            img=load_img(os.path.join(folder,image), target_size=(150,150))
            img=img_to_array(img)
            img=img/255.0
            dataset.append((img,count))
        print(f'\rCompleted: {label}',end='')
        count+=1
    random.shuffle(dataset)
    X, y = zip(*dataset)
    
    return np.array(X),np.array(y)

In [ ]:
X, y = input_target_split(directory,labels)

Completed: rock

### Memisahkan gambar dan label menjadi train and test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.22, random_state=42)
print(np.unique(y_train,return_counts=True),np.unique(y_test,return_counts=True))

(array([0, 1, 2]), array([557, 589, 560])) (array([0, 1, 2]), array([155, 161, 166]))


### Augmentasi gambar

In [ ]:
datagen = ImageDataGenerator(horizontal_flip=True,
                             vertical_flip=True,
                             rotation_range=20,
                             zoom_range=0.2,
                             width_shift_range = 0.2,
                             height_shift_range = 0.2,
                             shear_range=0.1,
                             fill_mode="nearest")

testgen = ImageDataGenerator()

datagen.fit(X_train)
testgen.fit(X_test)

In [ ]:
y_train = np.eye(nb)[y_train]
y_test = np.eye(nb)[y_test]

### CNN Model

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=(150,150,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(2, 2))
model.add(Flatten())
model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

### Summary

In [ ]:
# compile model dengan 'adam' optimizer loss function 'categorical_crossentropy' 
model.compile(optimizer = tf.keras.optimizers.Adam(lr = 0.001), loss = 'categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 41472)             0         
                                                                 
 dense_8 (Dense)             (None, 512)              

### Fitting dengan epochs 10

In [ ]:
model.fit_generator(datagen.flow(X_train,y_train,batch_size=32),
                    validation_data=testgen.flow(X_test,y_test,batch_size=32),
                    epochs=10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/10
54/54 [==============================] - 59s 1s/step - loss: 1.3422 - accuracy: 0.3880 - val_loss: 0.9302 - val_accuracy: 0.5809
Epoch 2/10
54/54 [==============================] - 58s 1s/step - loss: 0.7988 - accuracy: 0.6542 - val_loss: 0.5432 - val_accuracy: 0.8174
Epoch 3/10
54/54 [==============================] - 57s 1s/step - loss: 0.5879 - accuracy: 0.7749 - val_loss: 0.2891 - val_accuracy: 0.9129
Epoch 4/10
54/54 [==============================] - 56s 1s/step - loss: 0.4194 - accuracy: 0.8423 - val_loss: 0.1968 - val_accuracy: 0.9440
Epoch 5/10
54/54 [==============================] - 57s 1s/step - loss: 0.3524 - accuracy: 0.8658 - val_loss: 0.3939 - val_accuracy: 0.8071
Epoch 6/10
54/54 [==============================] - 57s 1s/step - loss: 0.2695 - accuracy: 0.9074 - val_loss: 0.1050 - val_accuracy: 0.9772
Epoch 7/10
54/54 [==============================] - 56s 1s/step - loss: 0.2131 - accuracy: 0.9308 - val_loss: 0.1254 - val_accuracy: 0.9564
Epoch 8/10
54/54 [==

### Classification Report Model

In [ ]:
y_pred=model.predict(X_test)
pred=np.argmax(y_pred,axis=1)
ground = np.argmax(y_test,axis=1)
from sklearn.metrics import classification_report

print(classification_report(ground,pred))

              precision    recall  f1-score   support

           0       0.99      0.94      0.96       155
           1       0.98      0.98      0.98       161
           2       0.95      1.00      0.98       166

    accuracy                           0.97       482
   macro avg       0.97      0.97      0.97       482
weighted avg       0.97      0.97      0.97       482



### Pengujian

In [ ]:
from google.colab import files
from keras.preprocessing import image

reverse_mapping={0:'paper',2:'rock',1:'scissors'}

def mapper(value):
    return reverse_mapping[value]

uploaded = files.upload()
for fn in uploaded.keys():
  img = image.load_img(fn, target_size=(150,150))
  x = image.img_to_array(img)
  img=img_to_array(img) 
  img=img/255.0

  prediction_image=np.array(img)
  prediction_image= np.expand_dims(img, axis=0)
  prediction=model.predict(prediction_image)
  value=np.argmax(prediction)
  move_name=mapper(value)
  print("\nPREDICTION IS {}".format(move_name))


Saving dZ6rscIYTbmlwVZE.png to dZ6rscIYTbmlwVZE.png

PREDICTION IS rock
